In [1]:
import pandas as pd
import warnings
import tkinter as tk
import os
from tkinter import messagebox
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
fecha = f"{año}{str(mes).zfill(2)}" # 202401

reporte_condonacion = f"Reporte de Condonación 17.10.2024.xlsx" # Modificar
base_final = f"Base_Final_{fecha}.xlsx"
base_final_path = os.path.abspath(base_final)

print("Reporte de Condonación:", reporte_condonacion)
print("Base Final:", base_final_path)

Reporte de Condonación: Reporte de Condonación 17.10.2024.xlsx
Base Final: c:\Users\miria\Documents\Mirko\BBVA\GitHub\analisis-condonaciones\Base_Final_202410.xlsx


In [2]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [3]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

result = messagebox.askquestion("Confirmación", "¿Cargar reporte de condonación?", icon='warning')
if result == 'yes':
    df_base = pd.read_excel(reporte_condonacion)

In [96]:
df_base_test = df_base.copy()
print(df_base_test.shape)

df_base_test.columns = clean_columns(df_base_test.columns)

cols_required = [
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE', 
    'CC', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA', 
    'MARCA_TEMPORAL', 
    'FECHA_DE_REVISIÓN', 
    '¿REVISIÓN_CONFORME?', 
    'FECHA_DE_EJECUCIÓN', 
    'EJECUCIÓN'
    ]

df_base_test = df_base_test[cols_required]
df_base_test.rename(columns={
    'MARCA_TEMPORAL': 'FECHA_INGRESO', 
    'SELECCIONAR_LA_AGENCIA_DE_COBRANZA_/_UNIDAD_CORRESPONDIENTE': 'AGENCIA', 
    'N°_DEL_CONTRATO_DE_CLIENTE_/_CÓDIGO_UGA': 'CONTRATO', 
    'FECHA_DE_EJECUCIÓN': 'FECHA_EJECUCION', 
    '¿REVISIÓN_CONFORME?': 'CONFORME', 
    'FECHA_DE_REVISIÓN': 'FECHA_REVISION', 
    'EJECUCIÓN': 'EJECUCION'
    }, inplace=True)

df_base_test['AGENCIA'] = df_base_test['AGENCIA'].str.upper()
df_base_test['EJECUCION'] = df_base_test['EJECUCION'].str.upper()
df_base_test['CONFORME'] = df_base_test['CONFORME'].str.upper()
df_base_test['CC'] = df_base_test['CC'].astype('Int64').astype(str).str.zfill(8)
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].astype('Int64').astype(str).str.zfill(18)

df_base_test['FECHA_INGRESO'] = pd.to_datetime(df_base_test['FECHA_INGRESO'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
df_base_test['FECHA_REVISION'] = pd.to_datetime(df_base_test['FECHA_REVISION'], format='%d-%m-%Y', errors='coerce')
df_base_test['FECHA_EJECUCION'] = pd.to_datetime(df_base_test['FECHA_EJECUCION'], format='%d-%m-%Y', errors='coerce')

df_base_test['FECHA_INGRESO'] = df_base_test['FECHA_INGRESO'].dt.strftime('%d-%m-%Y %H:%M:%S')
df_base_test['FECHA_REVISION'] = df_base_test['FECHA_REVISION'].dt.strftime('%d-%m-%Y')
df_base_test['FECHA_EJECUCION'] = df_base_test['FECHA_EJECUCION'].dt.strftime('%d-%m-%Y')

df_base_test = df_base_test[df_base_test['AGENCIA'].isin(['RJ ABOGADOS', 'CLASA', 'MORNESE'])]
df_base_test['AGENCIA'] = df_base_test['AGENCIA'].replace('RJ ABOGADOS', 'RJ')

df_base_test = df_base_test[df_base_test['FECHA_REVISION'].notnull()]
df_base_test = df_base_test[~((df_base_test['EJECUCION'] == 'SI') & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].isnull()) & (df_base_test['FECHA_EJECUCION'].notnull()))]
df_base_test = df_base_test[~((df_base_test['EJECUCION'].notnull()) & (df_base_test['FECHA_EJECUCION'].isnull()))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'NO') & (df_base_test['EJECUCION'] == 'SI'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'] == 'NO'))]
df_base_test = df_base_test[~((df_base_test['CONFORME'] == 'SI') & (df_base_test['EJECUCION'].isnull()))]
df_base_test['EJECUCION'].fillna('NO', inplace=True)

df_base_test['FECHA_INGRESO'] = df_base_test['FECHA_INGRESO'].str.split(' ').str[0]
df_base_test['FECHA_INGRESO'] = pd.to_datetime(df_base_test['FECHA_INGRESO'], format='%d-%m-%Y', errors='coerce')
df_base_test['FECHA_REVISION'] = pd.to_datetime(df_base_test['FECHA_REVISION'], format='%d-%m-%Y', errors='coerce')
df_base_test['FECHA_EJECUCION'] = pd.to_datetime(df_base_test['FECHA_EJECUCION'], format='%d-%m-%Y', errors='coerce')

df_base_test.sort_values(by=['CONTRATO', 'FECHA_INGRESO'], ascending=True, inplace=True)
df_base_test.reset_index(drop=True, inplace=True)

print(df_base_test.shape)
df_base_test.head(5)

(55966, 13)
(43244, 8)


,AGENCIA,CC,CONTRATO,FECHA_INGRESO,FECHA_REVISION,CONFORME,FECHA_EJECUCION,EJECUCION
0,MORNESE,26910745,000000000000755211,2024-06-04,2024-06-11,NO,NaT,NO
1,RJ,24204625,001100025000043500,2023-09-07,2023-09-08,SI,2023-09-08,SI
2,MORNESE,26217657,001100025000080309,2023-07-24,2023-07-26,SI,2023-07-26,SI
3,CLASA,27608713,001100025000130535,2023-03-31,2023-04-03,SI,2023-04-03,SI
4,MORNESE,24007029,001100025000163735,2024-08-02,2024-08-14,SI,2024-08-14,SI


In [97]:
df_base_test[['CONFORME', 'EJECUCION']].value_counts(dropna=False)

CONFORME  EJECUCION
SI        SI           40784
NO        NO            2460
Name: count, dtype: int64

In [110]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday

class Feriados(AbstractHolidayCalendar):
    rules = [
        Holiday('Año Nuevo', month=1, day=1),
        Holiday('Día del trabajo', month=5, day=1),
        Holiday('Día de independencia', month=7, day=28),
        Holiday('Santa Rosa de Lima', month=8, day=30),
        Holiday('Batalla de Angamos', month=10, day=8),
        Holiday('Día de todos los santos', month=11, day=1),
        Holiday('Inmaculada Concepción', month=12, day=8),
        Holiday('Navidad', month=12, day=25),
    ]

def calcular_dias_habiles(df: pd.DataFrame) -> pd.DataFrame:
    resultados = []
    
    df_agrupado = df.groupby('CONTRATO')
    for contrato, grupo in df_agrupado:
        primer_ingreso = None
        ultima_ejecucion = None
        agencia = None
        reprocesos = 0
        for _, row in grupo.iterrows():
            if primer_ingreso is None:
                primer_ingreso = row['FECHA_INGRESO']
            if row['EJECUCION'] == 'SI':
                ultima_ejecucion = row['FECHA_EJECUCION']
                agencia = row['AGENCIA']
                break
            reprocesos += 1
        if primer_ingreso and ultima_ejecucion:
            feriados = Feriados().holidays(primer_ingreso, ultima_ejecucion).to_pydatetime().tolist()
            feriados = [d.date() for d in feriados]
            dias_habiles = np.busday_count(primer_ingreso.date(), ultima_ejecucion.date(), holidays=feriados)
            resultados.append({
                'CC': row['CC'],
                'CONTRATO': contrato,
                'AGENCIA': agencia,
                'FECHA_INGRESO': primer_ingreso,
                'FECHA_EJECUCION': ultima_ejecucion,
                'DIAS_HABILES': dias_habiles,
                'REPROCESO': reprocesos
            })
    
    df_result = pd.DataFrame(resultados)
    return df_result

In [111]:
df_dias_habiles = calcular_dias_habiles(df_base_test)
df_dias_habiles

,CC,CONTRATO,AGENCIA,FECHA_INGRESO,FECHA_EJECUCION,DIAS_HABILES,REPROCESO
0,24204625,001100025000043500,RJ,2023-09-07,2023-09-08,1,0
1,26217657,001100025000080309,MORNESE,2023-07-24,2023-07-26,2,0
2,27608713,001100025000130535,CLASA,2023-03-31,2023-04-03,1,0
3,24007029,001100025000163735,MORNESE,2024-08-02,2024-08-14,8,0
4,26815640,001100025000235167,MORNESE,2023-04-01,2023-04-04,1,0
...,...,...,...,...,...,...,...
40149,27384208,001109829600078388,CLASA,2024-02-28,2024-03-01,2,0
40150,22314019,001109915000001997,RJ,2024-09-04,2024-09-13,7,0
40151,27906642,001109995000091388,CLASA,2024-03-22,2024-04-01,6,0
40152,24065104,001109995000131452,MORNESE,2024-01-12,2024-01-22,6,0


In [112]:
df_dias_habiles.to_excel(base_final_path, index=False)
os.startfile(base_final_path)